In [124]:
import pandas as pd
import numpy as np
from requests import get
from pandas import json_normalize
import time
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample

In [186]:
modeling_data = pd.read_csv('/Users/patricknaylor/Desktop/CA_traffic/Data/selected_columns.csv')

In [187]:
print(modeling_data.nunique())

latitude                           281296
longitude                          315729
statewide_vehicle_type_at_fault        15
alcohol_involved                        2
truck_collision                         2
motorcycle_collision                    2
bicycle_collision                       2
pedestrian_collision                    2
lighting                                5
road_surface                            5
hit_and_run                             3
party_count                            14
injured_victims                        27
killed_victims                          7
location_type                           3
county_location                        58
population                              8
county_location_population             58
dtype: int64


In [188]:
def absolute_maximum_norm(column):
    return column / column.abs().max()

norm_columns = ['latitude', 'longitude', 'county_location_population']

for col in norm_columns:
    modeling_data[col] = absolute_maximum_norm(modeling_data[col])


In [189]:
modeling_data = pd.get_dummies(modeling_data, columns = ['alcohol_involved', 'statewide_vehicle_type_at_fault', 'truck_collision', 'motorcycle_collision', 'bicycle_collision', 'lighting', 'road_surface', 'hit_and_run', 'location_type',  'population', 'county_location'])


In [190]:
modeling_data['mortality_rate'] = modeling_data['killed_victims']/modeling_data['party_count']
modeling_data['injury_rate'] = modeling_data['injured_victims']/modeling_data['party_count']
modeling_data['is_injured'] = modeling_data['injured_victims'] != 0
modeling_data['is_injured'] = modeling_data['is_injured'].astype(int)

In [191]:
columns_list = list(modeling_data.columns)
removals = ['injured_victims', 'killed_victims',  'mortality_rate', 'injury_rate', 'is_injured', 'latitude', 'longitude']
for rem in removals:
    columns_list.remove(rem)
model_x = np.array(modeling_data[columns_list])
model_y = np.array(modeling_data['is_injured'])

In [192]:
print(model_x[0,:])

[0.         2.         0.01558083 0.         1.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.         0.         0.
 0.         0.         1.         0.         1.         0.
 1.         0.         0.         0.         0.         1.
 0.         0.         1.         0.         0.         0.
 0.         1.         0.         1.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         1.        ]


In [193]:
x_oversample, y_oversample = resample(model_x[model_y == 0], model_y[model_y == 0], replace=True, n_samples=model_x[model_y ==1].shape[0])
X = np.vstack((model_x[model_y == 1], x_oversample))
y = np.hstack((model_y[model_y == 1], y_oversample))

In [194]:
print(np.shape(X))
print(np.shape(y))

(485180, 50)
(485180,)


In [195]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [196]:
print(np.mean((y_train)))

0.49960316239579167


In [197]:
reg = SGDClassifier(loss='log_loss')
reg.fit(X_train, y_train)
pred_train = reg.predict(X_train)
pred_test = reg.predict(X_test)


In [198]:
print(classification_report(y_train, pred_train))
print(classification_report(y_test, pred_test))
print(1-np.mean(y_test))

              precision    recall  f1-score   support

           0       0.63      0.43      0.51    162664
           1       0.57      0.75      0.65    162406

    accuracy                           0.59    325070
   macro avg       0.60      0.59      0.58    325070
weighted avg       0.60      0.59      0.58    325070

              precision    recall  f1-score   support

           0       0.63      0.43      0.51     79926
           1       0.57      0.75      0.65     80184

    accuracy                           0.59    160110
   macro avg       0.60      0.59      0.58    160110
weighted avg       0.60      0.59      0.58    160110

0.4991943039160577
